# 지표 정리

### 기본 세팅

In [1]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage

import google.generativeai as genai

import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')



now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

## 데이터 

In [2]:

sql = f"""
select 
*
from `{project_id}.{dataset_id}.kor_ticker_list`
order by rank asc
"""

# 데이터 조회 쿼리 실행 결과
query_job = client.query(sql)

# 데이터프레임 변환
kor_ticker_list = query_job.to_dataframe()

In [3]:
kor_ticker_list

,ticker,corp_name,market,rank
0,005930,삼성전자,KOSPI,1
1,247540,에코프로비엠,KOSDAQ,1
2,000660,SK하이닉스,KOSPI,2
3,086520,에코프로,KOSDAQ,2
4,373220,LG에너지솔루션,KOSPI,3
...,...,...,...,...
2650,438580,엔에이치스팩25호,KOSDAQ,1698
2651,413600,키움제6호스팩,KOSDAQ,1699
2652,457940,에스케이증권제10호스팩,KOSDAQ,1700
2653,032685,소프트센우,KOSDAQ,1701


In [4]:

# ticker_list = pd.read_csv( './data_crawler/kor_ticker_list/kor_ticker_list.csv')

# 코스피 지표
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kospi.parquet')
kor_stock_ohlcv_kospi = t.to_pandas()

# 코스피 매수/매도 지표
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kospi.parquet')
kor_stock_ohlcv_anal_kospi = t.to_pandas()

# # 코스피 주식별 매수매도 지표
# t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kospi.parquet')
# buy_sell_count_kospi = t.to_pandas()

# 코스닥 지표
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kospi.parquet')
kor_stock_ohlcv_kosdaq = t.to_pandas()

# 코스닥 매수/매도 지표
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kosdaq.parquet')
kor_stock_ohlcv_anal_kosdaq = t.to_pandas()

# # 코스닥 주식별 매수매도 지표
# t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kosdaq.parquet')
# buy_sell_count_kosdaq = t.to_pandas()


# 인덱스 지표
t = pq.read_table('data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_cleaning.parquet')
kor_index_ohlcv_cleaning = t.to_pandas()

# 인덱스 매수/매도 지표
t = pq.read_table('data_crawler/cleaning/kor_index_ohlcv/kor_index_ohlcv_anal_cleaning.parquet')
kor_index_ohlcv_anal_cleaning = t.to_pandas()

# 인덱스 PBR
t = pq.read_table('data_crawler/kor_index_code_fundamental/kor_index_code_fundamental.parquet')
kor_index_code_fundamental = t.to_pandas()


index_code_master = pd.read_csv('data_crawler/index_code_master/index_code_master.csv', dtype = {'ticker':str, 
                                                                                                'index_code':str})

not_sectors = ["1002","1003","1004","1028","1034","1035","1150","1151",
           "1152","1153","1154","1155","1156","1157","1158","1159",
           "1160","1167","1182","1224","1227","1232","1244","1894",
           "2002","2003","2004","2181","2182","2183","2184","2189",
           "2203","2212","2213","2214","2215","2216","2217","2218"]

index_code_master = index_code_master[~index_code_master['index_code'].isin(not_sectors)].reset_index(drop = True)



## 상황 단일주가 (삼성전자

- trading 추가

### 코스피 주가 등락률

In [5]:

### 코스피

# df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] > '2023-10-01'].groupby("ticker")['date'].max().reset_index()
df1_set =  kor_stock_ohlcv_kospi.groupby("ticker")['date'].max().reset_index()
df1_set = df1_set[['ticker', 'date']]
df1_set = pd.merge(df1_set, kor_stock_ohlcv_kospi[['ticker', 'date', 'close', 'corp_name']],
              on = ['ticker', 'date'], 
              how = 'left')

diff_date_list = [30, 90, 180, 240, 365]
for diff_date in diff_date_list:
# diff_date = 240
    now = datetime.now()
    now = now + timedelta(days=-diff_date)
    set_date = now.strftime('%Y-%m-%d')
    df1 =  kor_stock_ohlcv_kospi[kor_stock_ohlcv_kospi['date'] > set_date].groupby("ticker")['date'].min().reset_index()

    
    df1 = df1[['ticker','date']]
    df1 = pd.merge(df1, kor_stock_ohlcv_kospi[['ticker', 'date', 'close']],
                  on = ['ticker', 'date'], 
                  how = 'left')
    
    df1.columns = ['ticker', f'date_{diff_date}', f'close_{diff_date}']

    df1_set = pd.merge(df1_set, df1, 
                      on = 'ticker', 
                      how = 'left')

    df1_set[f'per_{diff_date}'] = (df1_set['close'] - df1_set[f'close_{diff_date}']) / df1_set[f'close_{diff_date}'] * 100


df1_set_2 = df1_set[['date','ticker','per_30', 'per_90', 'per_180', 'per_240', 'per_365', 'corp_name']]

df1_set_3 = pd.melt(df1_set_2, 
        id_vars= ['date', 'ticker', 'corp_name'], 
        value_vars=['per_30', 'per_90', 'per_180', 'per_240','per_365'])    
stock_ratio_per = df1_set_3.sort_values(by = ['ticker'])
stock_ratio_per.head()



,date,ticker,corp_name,variable,value
0,2024-01-16 00:00:00+00:00,000020,동화약품,per_30,4.727646
2859,2024-01-16 00:00:00+00:00,000020,동화약품,per_240,9.806034
1906,2024-01-16 00:00:00+00:00,000020,동화약품,per_180,-0.585366
3812,2024-01-16 00:00:00+00:00,000020,동화약품,per_365,10.400867
953,2024-01-16 00:00:00+00:00,000020,동화약품,per_90,5.815161


#### 주가 등락률 순위

In [6]:
df_per_total = pd.DataFrame()
per_set = ['per_30', 'per_90', 'per_180', 'per_240', 'per_365']
ascending_list = [True, False]

for per_value in per_set:
    for ascending_value in ascending_list:
        df_per =  df1_set_2.sort_values(by = per_value, ascending = ascending_value).head()
        df_per['type'] = per_value
        df_per['rank'] = range(1,6)
        df_per['음/양'] = ascending_value
        df_per_total = pd.concat([df_per_total, df_per])

df_per_total = df_per_total.drop_duplicates()     
df_per_total = df_per_total.reset_index(drop = True)



### 주가 보조지표 매수/매도 

In [7]:
# dfdf = kor_stock_ohlcv_anal_kospi[kor_stock_ohlcv_anal_kospi['ticker'] == ticker_nm].tail(1)

In [8]:
date_nm = max(kor_stock_ohlcv_anal_kospi['date'])
dfdf = kor_stock_ohlcv_anal_kospi[kor_stock_ohlcv_anal_kospi['date'] == date_nm]

In [9]:
# pd.wide_to_long(dfdf, stubnames='ht', i=['date', 'ticker','corp_name','market','close'], j='age')
dfdf2 = pd.melt(dfdf, 
        id_vars= ['date', 'ticker', 'corp_name'], 
        value_vars=['5_20_cross', '20_60_cross', 'array', 'Bollinger_band','MACD','RSI'])
stock_indicator = dfdf2[dfdf2['value'] != '-']
stock_indicator

,date,ticker,corp_name,variable,value
5,2024-01-16 00:00:00+00:00,068270,셀트리온,5_20_cross,데드크로스(매도)
32,2024-01-16 00:00:00+00:00,022100,포스코DX,5_20_cross,데드크로스(매도)
34,2024-01-16 00:00:00+00:00,326030,SK바이오팜,5_20_cross,데드크로스(매도)
41,2024-01-16 00:00:00+00:00,361610,SK아이이테크놀로지,5_20_cross,데드크로스(매도)
43,2024-01-16 00:00:00+00:00,161390,한국타이어앤테크놀로지,5_20_cross,골든크로스(매수)
...,...,...,...,...,...
5269,2024-01-16 00:00:00+00:00,000157,두산2우B,RSI,RSI 하단 상향 돌파(매수)
5284,2024-01-16 00:00:00+00:00,145270,케이탑리츠,RSI,RSI 하단 상향 돌파(매수)
5291,2024-01-16 00:00:00+00:00,002820,SUN&L,RSI,RSI 상단 하향돌파(매도)
5298,2024-01-16 00:00:00+00:00,084670,동양고속,RSI,RSI 하단 상향 돌파(매수)


### 코스닥 주가 등락률

In [10]:


### 코스닥

# df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] > '2023-10-01'].groupby("ticker")['date'].max().reset_index()
df1_set =  kor_stock_ohlcv_kosdaq.groupby("ticker")['date'].max().reset_index()
df1_set = df1_set[['ticker', 'date']]
df1_set = pd.merge(df1_set, kor_stock_ohlcv_kosdaq[['ticker', 'date', 'close', 'corp_name']],
              on = ['ticker', 'date'], 
              how = 'left')

diff_date_list = [30, 90, 180, 240, 365]
for diff_date in diff_date_list:
# diff_date = 240
    now = datetime.now()
    now = now + timedelta(days=-diff_date)
    set_date = now.strftime('%Y-%m-%d')
    df1 =  kor_stock_ohlcv_kosdaq[kor_stock_ohlcv_kosdaq['date'] > set_date].groupby("ticker")['date'].min().reset_index()

    
    df1 = df1[['ticker','date']]
    df1 = pd.merge(df1, kor_stock_ohlcv_kosdaq[['ticker', 'date', 'close']],
                  on = ['ticker', 'date'], 
                  how = 'left')
    
    df1.columns = ['ticker', f'date_{diff_date}', f'close_{diff_date}']

    df1_set = pd.merge(df1_set, df1, 
                      on = 'ticker', 
                      how = 'left')

    df1_set[f'per_{diff_date}'] = (df1_set['close'] - df1_set[f'close_{diff_date}']) / df1_set[f'close_{diff_date}'] * 100


df1_set_2 = df1_set[['date','ticker','per_30', 'per_90', 'per_180', 'per_240', 'per_365', 'corp_name']]

df1_set_3 = pd.melt(df1_set_2, 
        id_vars= ['date', 'ticker', 'corp_name'], 
        value_vars=['per_30', 'per_90', 'per_180', 'per_240','per_365'])    
kosdaq_stock_ratio_per = df1_set_3.sort_values(by = ['ticker'])
kosdaq_stock_ratio_per.head()

,date,ticker,corp_name,variable,value
0,2024-01-16 00:00:00+00:00,000020,동화약품,per_30,4.727646
2859,2024-01-16 00:00:00+00:00,000020,동화약품,per_240,9.806034
1906,2024-01-16 00:00:00+00:00,000020,동화약품,per_180,-0.585366
3812,2024-01-16 00:00:00+00:00,000020,동화약품,per_365,10.400867
953,2024-01-16 00:00:00+00:00,000020,동화약품,per_90,5.815161


#### 주가 등락률 순위 

In [11]:


df_per_total = pd.DataFrame()
per_set = ['per_30', 'per_90', 'per_180', 'per_240', 'per_365']
ascending_list = [True, False]

for per_value in per_set:
    for ascending_value in ascending_list:
        df_per =  df1_set_2.sort_values(by = per_value, ascending = ascending_value).head()
        df_per['type'] = per_value
        df_per['rank'] = range(1,6)
        df_per['음/양'] = ascending_value
        df_per_total = pd.concat([df_per_total, df_per])

df_per_total = df_per_total.drop_duplicates()     
kosdaq_df_per_total = df_per_total.reset_index(drop = True)

### 주가 보조지표 매수/매도

In [12]:

date_nm = max(kor_stock_ohlcv_anal_kosdaq['date'])
dfdf = kor_stock_ohlcv_anal_kosdaq[kor_stock_ohlcv_anal_kosdaq['date'] == date_nm]


# pd.wide_to_long(dfdf, stubnames='ht', i=['date', 'ticker','corp_name','market','close'], j='age')
dfdf2 = pd.melt(dfdf, 
        id_vars= ['date', 'ticker', 'corp_name'], 
        value_vars=['5_20_cross', '20_60_cross', 'array', 'Bollinger_band','MACD','RSI'])
kosdaq_stock_indicator = dfdf2[dfdf2['value'] != '-']

kosdaq_stock_indicator.head(3)

,date,ticker,corp_name,variable,value
20,2024-01-16 00:00:00+00:00,222800,심텍,5_20_cross,데드크로스(매도)
22,2024-01-16 00:00:00+00:00,237690,에스티팜,5_20_cross,데드크로스(매도)
40,2024-01-16 00:00:00+00:00,189300,인텔리안테크,5_20_cross,데드크로스(매도)


### 인덱스 등락률

In [40]:
# kor_index_ohlcv_cleaning_sample = kor_index_ohlcv_cleaning[kor_index_ohlcv_cleaning['index_code'].isin(ticker_index_code_df['index_code'])]

In [ ]:
# df1_set =  ohlcv_df_raw[ohlcv_df_raw['date'] > '2023-10-01'].groupby("ticker")['date'].max().reset_index()
df1_set =  kor_index_ohlcv_cleaning.groupby("index_code")['date'].max().reset_index()
df1_set = df1_set[['index_code', 'date']]
df1_set = pd.merge(df1_set, kor_index_ohlcv_cleaning[['index_code', 'date', 'close', 'index_code_nm']],
              on = ['index_code', 'date'], 
              how = 'left')



diff_date_list = [30, 90, 180, 240, 365]
for diff_date in diff_date_list:
# diff_date = 240
    now = datetime.now()
    now = now + timedelta(days=-diff_date)
    set_date = now.strftime('%Y-%m-%d')
    df1 =  kor_index_ohlcv_cleaning[kor_index_ohlcv_cleaning['date'] > set_date].groupby("index_code")['date'].min().reset_index()

    
    df1 = df1[['index_code','date']]
    df1 = pd.merge(df1, kor_index_ohlcv_cleaning[['index_code', 'date', 'close']],
                  on = ['index_code', 'date'], 
                  how = 'left')
    
    df1.columns = ['index_code', f'date_{diff_date}', f'close_{diff_date}']

    df1_set = pd.merge(df1_set, df1, 
                      on = 'index_code', 
                      how = 'left')

    df1_set[f'per_{diff_date}'] = (df1_set['close'] - df1_set[f'close_{diff_date}']) / df1_set[f'close_{diff_date}'] * 100

df1_set_2 = df1_set[['date','index_code','per_30', 'per_90', 'per_180', 'per_240', 'per_365', 'index_code_nm']]


df1_set_3 = pd.melt(df1_set_2, 
        id_vars= ['date', 'index_code', 'index_code_nm'], 
        value_vars=['per_30', 'per_90', 'per_180', 'per_240','per_365'])    
index_ratio_per = df1_set_3.sort_values(by = ['index_code'])
index_ratio_per.head()


#### 인덱스 등락률 순위

In [ ]:

df_per_total = pd.DataFrame()
per_set = ['per_30', 'per_90', 'per_180', 'per_240', 'per_365']
ascending_list = [True, False]

for per_value in per_set:
    for ascending_value in ascending_list:
        df_per =  df1_set_2.sort_values(by = per_value, ascending = ascending_value).head()
        df_per['type'] = per_value
        df_per['rank'] = range(1,6)
        df_per['음/양'] = ascending_value
        df_per_total = pd.concat([df_per_total, df_per])

df_per_total = df_per_total.drop_duplicates()     
df_per_total = df_per_total.reset_index(drop = True)

In [ ]:
df_per_total.head(2)

### 인덱스 보조지표 매수/매도

In [44]:
# kor_index_ohlcv_anal_cleaning_df = kor_index_ohlcv_anal_cleaning[kor_index_ohlcv_anal_cleaning['index_code'].isin(ticker_index_code_df['index_code'])]


date_nm = max(kor_index_ohlcv_anal_cleaning['date'])
kor_index_ohlcv_anal_cleaning_df_22 = kor_index_ohlcv_anal_cleaning[kor_index_ohlcv_anal_cleaning['date'] == date_nm]

In [45]:
# pd.wide_to_long(dfdf, stubnames='ht', i=['date', 'ticker','corp_name','market','close'], j='age')
kor_index_ohlcv_anal_cleaning_df_2 = pd.melt(kor_index_ohlcv_anal_cleaning_df_22, 
        id_vars= ['date', 'index_code', 'index_code_nm'], 
        value_vars=['5_20_cross', '20_60_cross', 'array', 'Bollinger_band','MACD','RSI'])

index_indicator = kor_index_ohlcv_anal_cleaning_df_2[kor_index_ohlcv_anal_cleaning_df_2['value'] != '-']

In [46]:
index_indicator.head()

,date,index_code,index_code_nm,variable,value
15,2024-01-05 00:00:00+00:00,1016,유통업,5_20_cross,데드크로스(매도)
32,2024-01-05 00:00:00+00:00,1154,코스피 200 에너지/화학,5_20_cross,데드크로스(매도)
34,2024-01-05 00:00:00+00:00,1156,코스피 200 금융,5_20_cross,데드크로스(매도)
39,2024-01-05 00:00:00+00:00,1167,코스피 200 중소형주,5_20_cross,데드크로스(매도)
70,2024-01-05 00:00:00+00:00,2072,일반전기전자,5_20_cross,골든크로스(매수)


### 최종

In [47]:
### 최종 
df_kospi = pd.concat([stock_ratio_per,stock_indicator])
df_kospi['type'] = 'stock'
df_kospi.columns = ['date', 'code', 'code_nm', '등락률기간', '등락률', 'type']

df_kosdaq = pd.concat([kosdaq_stock_ratio_per,kosdaq_stock_indicator])
df_kosdaq['type'] = 'stock'
df_kosdaq.columns = ['date', 'code', 'code_nm', '등락률기간', '등락률', 'type']

df_index = pd.concat([index_ratio_per,index_indicator])
df_index['type'] = 'index'
df_index.columns = ['date', 'code', 'code_nm', '등락률기간', '등락률', 'type']

df_total = pd.concat([df_kospi,df_kosdaq]).reset_index(drop = True)
df_total = pd.concat([df_total,df_index]).reset_index(drop = True)

df_total = df_total[['code','code_nm', '등락률기간', '등락률', 'type']]

In [48]:
df_total

,code,code_nm,등락률기간,등락률,type
0,000020,동화약품,per_30,7.605344,stock
1,000020,동화약품,per_240,12.823276,stock
2,000020,동화약품,per_180,3.051181,stock
3,000020,동화약품,per_365,12.098501,stock
4,000020,동화약품,per_90,8.049536,stock
...,...,...,...,...,...
11203,2031,금융,MACD,상향돌파(매수),index
11204,2056,음식료·담배,MACD,하향돌파(매도),index
11205,2181,코스닥 우량기업부,MACD,상향돌파(매수),index
11206,2189,코스닥 글로벌,MACD,상향돌파(매수),index


## gemini 

In [63]:
kospi_ticker_list = kor_ticker_list[kor_ticker_list['market'] == 'KOSPI']

In [64]:

# ticker_nm = '005930'
total_response_df = pd.DataFrame()
for ticker_nm in kospi_ticker_list['ticker'][:10]:
    ticker_index_code_df = index_code_master[index_code_master['ticker'] == ticker_nm].reset_index(drop = True)
    index_code_list = ticker_index_code_df['index_code']


    total_df = df_total[(df_total['code'] == ticker_nm) | (df_total['code'].isin(index_code_list))].reset_index(drop = True)
    total_df_1 = total_df[~total_df['등락률기간'].isin(['5_20_cross', '20_60_cross', 'array', 'Bollinger_band','MACD','RSI'])]
    total_df_2 = total_df[total_df['등락률기간'].isin(['5_20_cross', '20_60_cross', 'array', 'Bollinger_band','MACD','RSI'])]

    
    # date_nm = df3['date'][0].strftime('%Y-%m-%d')

    prompt = f"""
    - 날짜:{date_nm} 

    - 등락률
        - {total_df_1}

    - 매수/매도신호
        - {total_df_2}


    증권 보고서 형태로 설명식으로 요약해줘. 
    """

    response = model.generate_content(prompt)
#     print(response.text)
    
    response_df = pd.DataFrame({'ticker':ticker_nm, 
                 'response_msg':response.text}, index = [0])
    
    if not os.path.exists('data_crawler/total_response_df.csv'):
        response_df.to_csv('data_crawler/total_response_df.csv', index=False, mode='w')
    else:
        response_df.to_csv('data_crawler/total_response_df.csv', index=False, mode='a', header=False)
        
#     response_df.to_csv('total_response_df.csv', index = False)
    
    print(ticker_nm)
#     total_response_df = pd.concat([total_response_df, response_df])
    
    time.sleep(1)
#     print(ticker_nm)


# total_response_df.to_csv('total_response_df.csv', index = False)

005930
000660
373220
207940
005935
005490
005380
035420
000270
051910


In [54]:
total_df_1

,code,code_nm,등락률기간,등락률,type
0,00104K,CJ4우(전환),per_180,33.394161,stock
1,00104K,CJ4우(전환),per_90,16.586922,stock
2,00104K,CJ4우(전환),per_365,-0.948509,stock
3,00104K,CJ4우(전환),per_30,3.394625,stock
4,00104K,CJ4우(전환),per_240,0.274348,stock
5,00104K,CJ4우(전환),per_180,33.394161,stock
6,00104K,CJ4우(전환),per_90,16.586922,stock
7,00104K,CJ4우(전환),per_365,-0.948509,stock
8,00104K,CJ4우(전환),per_30,3.394625,stock
9,00104K,CJ4우(전환),per_240,0.274348,stock


In [51]:
dfdf= pd.read_csv('total_response_df.csv')

In [52]:
dfdf

,ticker,response_msg
0,095570,**증권 보고서**\n\n**날짜:** 2024년 1월 5일\n\n**주식 코드:*...
1,006840,"종목코드 006840 AK홀딩스는 90일 등락률이 -0.058928%, 365일 등..."
2,027410,**증권 보고서**\n\n**일자:** 2024-01-05 00:00:00+00:0...
3,282330,**증권 보고서**\n\n**날짜:** 2024-01-05\n\n**주식:** BG...
4,138930,**날짜:** 2024년 1월 5일 00:00:00+00:00\n\n**등락률**\...
5,001460,증권 보고서\n\n**날짜:** 2024-01-05 00:00:00+00:00\n\...
6,001465,**등락률 및 매수/매도 신호**\n\n**증권 코드:** 001465\n\n**증...
7,001040,**증권 보고서**\n\n**날짜:** 2024년 1월 5일\n\n**주식 코드:*...
8,079160,**증권 보고서**\n\n**날짜:** 2024년 1월 5일\n\n**1. 시장 동...
9,00104K,**증권 보고서**\n\n**날짜:** 2024-01-05 00:00:00+00:0...
